### Primal & Dual Simplex

In [37]:
import numpy as np

def phase2(A, b, c, basis):
    m, n = A.shape
    objective = 0
    while True:
        basis.sort()
        A_b_inverse = np.linalg.inv(A[:, basis])
        A_bar = A_b_inverse.dot(A)
        b_bar = A_b_inverse.dot(b)
        mask = np.ones(n, bool)
        mask[basis] = False
        solution = np.zeros(n)
        for i, j in enumerate(basis):
            solution[j] = b_bar[i]
        objective = solution.dot(c)
        c_n_bar = c[mask] - c[basis].dot(A_bar[:, mask])
        if c_n_bar.max() <= 0:
            break
        c_ind = np.zeros_like(c).astype(float)
        c_ind[mask] = c_n_bar.copy()
        r = np.argmax(c_ind)
        if A_bar[:, r].max() <= 0:
            return "Optimal Value is Infinity!"
        lamda = np.array([b_bar[i]/A_bar[i, r] if A_bar[i, r]>0 else float("inf") for i in range(m)])
        s = np.argmin(lamda)
        basis.remove(basis[s])
        basis.append(r)
    results = {f'x_{i+1}': solution[i] for i in range(n)}
    results['obj'] = objective
    results['basis'] = basis
    return results

def phase1(A, b):
    m, n = A.shape
    for i in range(m):
        if b[i]<0:
            A[i, :] = -A[i, :] 
            b[i] = -b[i]
    A_slack = np.concatenate([A, np.eye(m)], axis=1)
    basis = list(range(n, n+m))
    c = np.zeros(n+m)
    c[n:(n+m)] = -1
    results = phase2(A_slack, b, c, basis)
    if results['obj'] < 0:
        return "Infeasible!"
    return results['basis']

def PrimeSimplex(A, b, c):
    basis = phase1(A, b)
    if basis == "Infeasible!":
        return basis
    return phase2(A, b, c, basis)

In [38]:
c = np.array([7, 2, 0, 0, 0])
A = np.array([[-1, 2, 1, 0, 0], [5, 1, 0, 1, 0], [-2, -2, 0, 0, 1]])
b = np.array([4, 20, -7])                                      
PrimeSimplex(A, b, c)                                                         

{'x_1': 3.272727272727273,
 'x_2': 3.6363636363636367,
 'x_3': 0.0,
 'x_4': 0.0,
 'x_5': 6.81818181818182,
 'obj': 30.181818181818183,
 'basis': [0, 1, 4]}

In [39]:
def DualSimplex(A, b, c, basis):
    m, n = A.shape
    while True:
        basis.sort()
        A_b_inverse = np.linalg.inv(A[:, basis])
        b_bar = A_b_inverse.dot(b)
        if np.all(b_bar >= 0):
            solution = np.zeros(n).astype(float)
            for i, j in enumerate(basis):
                solution[j] = b_bar[i]
            objective = solution.dot(c)
            results = {f'x_{i+1}': solution[i] for i in range(n)}
            results['obj'] = objective
            results['basis'] = basis
            return results
        A_bar = A_b_inverse.dot(A)
        s = np.argmin(b_bar)
        if np.all(A_bar[s, :]>=0):
            return "Infeasible!"
        mask = np.ones(n, bool)
        mask[basis] = False
        c_n_bar = c[mask] - c[basis].dot(A_bar[:, mask])
        c_ind = np.zeros_like(c).astype(float)
        c_ind[mask] = c_n_bar.copy()
        r = np.argmin(np.array([c_ind[j]/A_bar[s, j] if A_bar[s, j]<0 else float('inf') for j in range(n)]))
        basis.remove(basis[s])
        basis.append(r)

In [40]:
DualSimplex(A, b, c, [0, 1, 4])

{'x_1': 3.272727272727273,
 'x_2': 3.6363636363636367,
 'x_3': 0.0,
 'x_4': 0.0,
 'x_5': 6.81818181818182,
 'obj': 30.181818181818183,
 'basis': [0, 1, 4]}

### Cutting Plane Algorithm